In [ ]:
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict
import os
from dotenv import load_dotenv
from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
import json
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START
load_dotenv()

BASE_URL = "http://127.0.0.1:1234/v1"

## Define state

In [ ]:
class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str


## Define tools

In [ ]:
tools = [TavilySearch(max_results=1)]

## Define agents

### Planner Agents

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain.chat_models import init_chat_model

class Plan(BaseModel):
    """Plan to follow in future"""

    steps: list[str] = Field(
        description="Different steps to follow, should be in sorted order"
    )

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given objective, come up with a simple step by step plan.
This plan should involve individual tasks, that if executed correctly will yield
the correct answer. Do not add any superfluous steps.
The result of the final step should be the final answer. Make sure that each step
has all the information needed - do not skip steps."""
        ),
        ("placeholder", "{messages}"),
    ]
)

planner = planner_prompt | init_chat_model(
    model="llama-3.2-3b-instruct",
    model_provider="openai",            # provider “openai” để dùng endpoint tương thích
    base_url=BASE_URL,  # URL server LM Studio
    api_key="lm-studio",                # có thể bất kỳ chuỗi nào
    temperature=0.7
).with_structured_output(Plan)

plan = planner.invoke({"input": "What is the weather in Vietnam?"})
print(plan)

### Executor agents

In [ ]:
model = ChatOpenAI(
    base_url=BASE_URL,
    api_key="not-needed",  # LM Studio không yêu cầu API key
    temperature=0,
)
prompt = "You are a helpful assistant."
agent_executor = create_react_agent(model, tools, prompt=prompt)

In [ ]:
agent_executor.invoke(
    {"messages": [("user", "How is the weather in Vietnam?")]})

### Replanner Agents

In [ ]:
from typing import Union

class Response(BaseModel):
    """Response to user."""

    response: str

"""
Trường action có thể là:
    - Response: trả lời luôn cho user (kết thúc).
Plan: lên/kế hoạch mới gồm các bước cần làm tiếp (nếu chưa đủ dữ kiện để trả lời hoặc task chưa xong).
Union dùng để cho phép output linh hoạt: hoặc trả lời user, hoặc lên kế hoạch mới.
"""
class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
                    "If you need to further use tools to get the answer, use Plan."
    )


replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan.
This plan should involve individual tasks, that if executed correctly will yield
the correct answer. Do not add any superfluous steps.
The result of the final step should be the final answer. Make sure that each step
has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to
the user, then respond with that. Otherwise, fill out the plan. Only add steps
to the plan that still NEED to be done. Do not return previously done steps
as part of the plan."""
)



replanner = replanner_prompt | init_chat_model(
    model="llama-3.2-3b-instruct",
    model_provider="openai",            # provider “openai” để dùng endpoint tương thích
    base_url=BASE_URL,  # URL server LM Studio
    api_key="lm-studio",                # có thể bất kỳ chuỗi nào
    temperature=0.7
).with_structured_output(Plan)

## Define node and edge

In [ ]:
from typing import Literal
from langgraph.graph import END

async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    agent_response = await agent_executor.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }

async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}

async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}

def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "agent"

## Build graph

In [ ]:
workflow = StateGraph(PlanExecute)
workflow.add_node("planner", plan_step)
workflow.add_node("agent", execute_step)
workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")
workflow.add_edge("planner", "agent")
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges('replan', should_end, ["agent", END])
app = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph(xray=True).draw_mermaid_png()))

## Inferences

In [ ]:
config = {"recursion_limit": 50}
inputs = {
    "input": "Solve for all real solutions x to the equation: sqrt(x -2) = 3"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

In [ ]:
from pydantic import BaseModel, Field
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent

class Plan(BaseModel):
    """Plan to follow in future"""
    steps: list[str] = Field(
        description="Các bước cần thực hiện, theo đúng thứ tự"
    )

In [38]:
from typing import Literal
from pydantic import BaseModel
from autogen_core.models import ChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage
from autogen_agentchat.ui import Console

# 1. Định nghĩa structured schema
class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]


# 2. Cấu hình client với model_info đầy đủ cho LM Studio
config = {
    "provider": "OpenAIChatCompletionClient",
    "config": {
        "model": "llama-3.2-3b-instruct",
        "base_url": "http://localhost:1234/v1",
        "api_key": "lm-studio",
        "model_info": {
            "name": "llama-3.2-3b-instruct",
            "family": "openai",
            "supports_tool_calling": False,
            "supports_json_mode": False,
            "structured_output": True   ,
            "json_output": True,
            "function_calling": False,
            "vision": False,
        }
    }
}

# 3. Load client từ config
model_client = ChatCompletionClient.load_component(config)

# 4. Tạo agent sử dụng structured output
agent = AssistantAgent(
    name="emotion_agent",
    model_client=model_client,
    system_message=(
        "Categorize the user input as happy, sad, or neutral. "
        "Respond ONLY with a valid JSON object matching this format:\n"
        "{\n"
        "  \"thoughts\": \"...\",\n"
        "  \"response\": \"happy\" | \"sad\" | \"neutral\"\n"
        "}"
    ),
    output_content_type=AgentResponse
)

# 5. Gửi và stream kết quả
result = await Console(agent.run_stream(task="I am feeling wonderful today!"))

# 6. Parse structured message
msg = result.messages[-1]
assert isinstance(msg, StructuredMessage)
parsed: AgentResponse = msg.content

print("🧠 Thoughts:", parsed.thoughts)
print("🙂 Response:", parsed.response)

# 7. Đóng client
await model_client.close()

---------- TextMessage (user) ----------
I am feeling wonderful today!


---------- StructuredMessage[AgentResponse] (emotion_agent) ----------
{"thoughts":"You're expressing positive emotions and feeling good!","response":"happy"}
🧠 Thoughts: You're expressing positive emotions and feeling good!
🙂 Response: happy
